In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import seaborn as sns
from torch import nn
import sys
sys.path.append('../')
from fun.models import *
sns.set_style("whitegrid")
from tqdm import tqdm
import pandas as pd
torch.manual_seed(42)
np.random.seed(42)



### Prepare dataset and split into valid-train set

In [ ]:
# WITHOUT AUGMENTATION
transform = transforms.Compose([
    transforms.ToTensor()
])

train_ds = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    download=True,
    transform=transform

)

test_ds = torchvision.datasets.CIFAR10(
    root="../data",
    train=False,
    download=True,
    transform=transform
)


    
    

In [ ]:
# WITH AUGMENTATION

transform_test = transforms.Compose([
    transforms.ToTensor()
])

transform_train = transforms.Compose([
    transforms.Resize((34, 34)), 
    transforms.RandomCrop(32),
    transforms.RandomRotation(degrees=15),
    transforms.RandomHorizontalFlip(0.4),
    transforms.ToTensor()
    ])

train_ds = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    download=True,
    transform=transform_train
)

test_ds = torchvision.datasets.CIFAR10(
    root="../data",
    train=False,
    download=True,
    transform=transform_test
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

def train(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    train_loss /= num_batches
    correct /= size
    print(f" Train Accuracy: {(100*correct):>0.1f}%, Train Avg loss {train_loss:>8f} \n")
    return correct, train_loss


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Accuracy: {(100*correct):>0.1f}%, Test Avg loss: {test_loss:>8f} \n")
    return correct, test_loss
    

## Saving model

In [ ]:
batch_size = 64
epochs = 60
learning_rate = 1e-3
dropout_rate = 0.2


train_dataloader = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True
    )
test_dataloader = DataLoader(
    test_ds, 
    batch_size=batch_size,
    shuffle=False
    )

net = Net_dropout_new(dropout_rate=dropout_rate).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)


data = np.array([[0]*4]*epochs)
net_SGD_augm_history = pd.DataFrame(data, columns = ["test_acc", "train_acc", "test_loss", "train_loss"])


for t in range(epochs):
    
    print(f"Epoch {t}\n-------------------------------")
    train_acc, train_loss = train(train_dataloader, net, criterion, optimizer)
    test_acc, test_loss = test(test_dataloader, net, criterion)
    
    net_SGD_augm_history.loc[t,"train_acc"] = train_acc        
    net_SGD_augm_history.loc[t,"test_acc"] = test_acc
    net_SGD_augm_history.loc[t,"train_loss"] = train_loss
    net_SGD_augm_history.loc[t,"test_loss"] = test_loss

    torch.save({
        'epoch': t,
        'model_state_dict': net.state_dict(),
        }, r"..\models\net_new_bs64_dr0_2_adam_augm\model_epoch_"+str(t)+".pt")


 